In [2]:
from flask import Flask , jsonify
app = Flask(__name__)


@app.route('/<string:name>/<string:trial>/<int:n>')
def home1(name,trial,n):
    from elasticsearch import Elasticsearch
    import json

    from sklearn.neighbors import KNeighborsClassifier
    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt
    from sklearn import datasets
    from sklearn.model_selection import train_test_split
    from sklearn import metrics
    import operator
    import math
    for i in range(0,3):
        if i==0:
            naturalsubject="A"
            subjectbasic=naturalsubject
            arrif=[85,86,85,78,77,75,74,73,72,74]
        elif i==1:
            happysubject="電機及資訊科系測量"
            subjectbasic=happysubject
            arrif=[74,75,73,74,76,75,74,76,78,84]
        else :
            sadsubject= trial
            subjectbasic=sadsubject
            arrif=[80,79,78,80,81,82,80,81,80,81]
        def get_querybasic():
            querybasic = {
                "query": {
                    "bool": {
                        "must": [
                            {
                            "term": {
                                "name": name
                            }
                        },
                            {
                            "term": {
                                "subject.keyword": {"value": subjectbasic}
                            }
                        }]
                    }
                }
            }
            return querybasic
        if __name__ == "__main__":
            es = Elasticsearch(hosts='140.137.41.81', port=9200)
            querybasic = get_querybasic()
            resultbasic = es.search(index='hh3', body=querybasic)
            resultbasic1=json.dumps(resultbasic, ensure_ascii=False)
            resultPulsebasic=json.loads(resultbasic1)
            size=resultPulsebasic['hits']['total']['value']
            if size>10:
                arrbasic=[0]*10
                arrbasic1=[0]*10
                arrbasic2=[0]*10
                for a in range(0,10):
                    pulse=(resultPulsebasic['hits']['hits'][a]['_source']['pulse'])
                    time=(resultPulsebasic['hits']['hits'][a]['_source']['timeForCompare'])        
                    arrbasic[a]=pulse,time
                arrbasic1=sorted(arrbasic,key=lambda s:s[1])
                for b in range(0,10):
                    arrbasic2[b]=arrbasic1[b][0]
                if i==0:
                    arrnatural2=arrbasic2
                elif i==1:
                    arrhappy2=arrbasic2
                elif i==2:
                    arrsad2=arrbasic2
            else:
                arrbasic=[0]*size
                arrbasic1=[0]*size
                arrbasic2=[0]*10
                for a in range(0,size):
                    pulse=(resultPulsebasic['hits']['hits'][a]['_source']['pulse'])
                    time=(resultPulsebasic['hits']['hits'][a]['_source']['timeForCompare'])        
                    arrbasic[a]=pulse,time
                arrbasic1=sorted(arrbasic,key=lambda s:s[1])
                for b in range(0,size):
                    arrbasic2[b]=arrbasic1[b][0]
                for c in range(size,10):
                    arrbasic2[c]=arrif[c]
                if i==0:
                    arrnatural2=arrbasic2
                elif i==1:
                    arrhappy2=arrbasic2
                elif i==2:
                    arrsad2=arrbasic2
    print(arrnatural2)
    print(arrhappy2)
    print(arrsad2)

    arrnaturaltext1=[0]*8
    arrhappytext1=[0]*8
    arrsadtext1=[0]*8
    for i in range (0,8):
        arrnaturaltext1[i]=[arrnatural2[i],arrnatural2[i+1],arrnatural2[i+2]]
        arrhappytext1[i]=[arrhappy2[i],arrhappy2[i+1],arrhappy2[i+2]]
        arrsadtext1[i]=[arrsad2[i],arrsad2[i+1],arrsad2[i+2]]
    print(arrnaturaltext1)
    print(arrhappytext1)
    print(arrsadtext1)
    arrhappycount=[0.0]*8
    arrsadcount=[0.0]*8    
    for z in range(0,8):
        print(z)
        naturalsize=3
        happysize=3
        sadsize=3
        #正常心率平均
        ave = 0.0
        for i in range(0,naturalsize):
            ave+=arrnaturaltext1[z][i]
        ave = ave/naturalsize

        print("happy心率特徵 :")

        #平均
        aveB = 0.0
        for i in range(0,happysize):
            aveB +=arrhappytext1[z][i]
        aveB = aveB/naturalsize
        print("平均: ",aveB)

        #DiffB1 一階差的平均值
        diffB1 = 0.0
        meanB1 = 0.0
        #X[n-1]-X[n]的總和
        for i in range(0,(happysize-1)):
            meanB1+=abs(arrhappytext1[z][i+1]-arrhappytext1[z][i])
        diffB1 = meanB1/(naturalsize-1)
        print("一階差: ",diffB1)

        #DiffB2 二階差的平均值
        diffB2 = 0.0
        meanB2 = 0.0
        #X[n+2]-X[n]的總和
        for i in range(0,(happysize-2)):
            meanB2 += abs(arrhappytext1[z][i+2]-arrhappytext1[z][i])
        diffB2 = meanB2/(happysize-2)
        print("二階差: ",diffB2)

        #range 心率變化範圍
        scopeB = 0
        scopemaxB = 0
        scopeminB = 110

        for i in range(0,happysize):
            #求最大心率
            if arrhappytext1[z][i]>scopemaxB:
                scopemaxB = arrhappytext1[z][i]
            if arrhappytext1[z][i]<scopeminB:
                scopeminB = arrhappytext1[z][i]
        scopeB = scopemaxB-scopeminB
        print("最大心率: ",scopemaxB)
        print("最小心率: ",scopeminB)
        print("心率變化範圍: ",scopeB)

        #H(X)資訊熵
        x = 0
        HB = 0
        arrpb = [0.0]*happysize
        for i in range(0,happysize):
            x=0 #初值為0
            #心率出現次數
            for j in range(0,happysize):
                if arrhappytext1[z][i]==arrhappytext1[z][j]:
                    x+=1
            #p[i]機率
            arrpb[i] = x*1.0/happysize
        for k in range(0,happysize):
            HB += -arrpb[k]*math.log(arrpb[k],2)
        print("資訊熵: ",HB)

        #Radrm 均方根
        radrmB = 0.0   
        meanrB = 0.0
        #相鄰值平方差的總和
        for i in range(0,(happysize-1)):
            radrmB += pow(arrhappytext1[z][i+1]-arrhappytext1[z][i],2)
        meanrB = radrmB/happysize
        print("均方根: ",meanrB)

        #標準化均值
        arrbb=[0.0]*happysize
        aveb = 0.0
        for i in range(0,happysize):
            arrbb[i] = arrhappytext1[z][i]-ave
            aveb += arrbb[i]
        aveb = aveb/happysize
        print("標準化均值: ",aveb)
        arrhappycount[z]=aveB,diffB1,diffB2,scopemaxB,scopeminB,scopeB,HB,meanrB,aveb
        print(arrhappycount)   
        print("subject心率特徵: ")

        #平均
        aveD = 0.0
        for i in range(0,sadsize):
            aveD +=arrsadtext1[z][i]
        aveD = aveD/sadsize
        print("平均: ",aveD)

        #DiffD1 一階差的平均值
        diffD1 = 0.0
        meanD1 = 0.0
        #X[n+1]-X[n]的總和
        for i in range(0,(sadsize-1)):
                meanD1 += abs(arrsadtext1[z][i+1]-arrsadtext1[z][i])
        diffD1 = meanD1/(sadsize-1)
        print("一階差: ",diffD1)

        #DiffD2 二階差的平均值
        diffD2 = 0.0
        meanD2 = 0.0
        #X[n+2]-X[n]的總和
        for i in range(0,(sadsize-2)):
            meanD2 += abs(arrsadtext1[z][i+2]-arrsadtext1[z][i])
        diffD2 = meanD2/(sadsize-2)
        print("二階差: ",diffD2)

        #range 心率變化範圍
        scopeD = 0
        scopemaxD = 0
        scopeminD = 110
        for i in range(0,sadsize):
            #求最大心率
            if arrsadtext1[z][i]>scopemaxD:
                scopemaxD = arrsadtext1[z][i]
            #求最小心率
            if arrsadtext1[z][i]<scopeminD:
                scopeminD = arrsadtext1[z][i]
        scopeD = scopemaxD-scopeminD
        print("最大心率: ",scopemaxD)
        print("最小心率: ",scopeminD)
        print("心率變化範圍: ",scopeD)

        #H(X)資訊熵
        x = 0
        HD = 0
        arrpd = [0.0]*sadsize
        for i in range(0,sadsize):
            x = 0 #初值為0 
            #心率出現次數
            for j in range(0,sadsize):
                if arrsadtext1[z][i]==arrsadtext1[z][j]:
                    x+=1
            arrpd[i] = x*1.0/sadsize
        for k in range(0,sadsize):
            HD += -arrpd[k]*math.log(arrpd[k],2)
        print("資訊熵: ",HD)

        #Radrm 均方根
        radrmD = 0.0
        meanrD = 0.0
        #相鄰值平方差的總和
        for i in range(0,(sadsize-1)):
            radrmD += pow(arrsadtext1[z][i+1]-arrsadtext1[z][i],2)
        meanrD = radrmD/sadsize
        print("均方根: ",meanrD)

        #標準化均值
        arrdd = [0.0]*sadsize
        aved = 0.0
        for i in range(0,sadsize):
            arrdd[i] = arrsadtext1[z][i] -ave
            aved +=arrdd[i]
        aved = aved/sadsize
        print("標準化均值: ",aved)
        arrsadcount[z]=aveD,diffD1,diffD2,scopemaxD,scopeminD,scopeD,HD,meanrD,aved
    print(arrhappycount)
    print(arrsadcount)
    arrX0=[0.0]*16
    arrX1=[0.0]*16
    arrX2=[0.0]*16
    arrX3=[0.0]*16
    arrX4=[0.0]*16
    arrX5=[0.0]*16
    arrX6=[0.0]*16
    arrX7=[0.0]*16
    arrX8=[0.0]*16
    for i in range(0,16):
        if i<8:
            arrX0[i]=arrhappycount[i][0]
            arrX1[i]=arrhappycount[i][1]
            arrX2[i]=arrhappycount[i][2]
            arrX3[i]=arrhappycount[i][3]
            arrX4[i]=arrhappycount[i][4]
            arrX5[i]=arrhappycount[i][5]
            arrX6[i]=arrhappycount[i][6]
            arrX7[i]=arrhappycount[i][7]
            arrX8[i]=arrhappycount[i][8]
        else:
            arrX0[i]=arrsadcount[i-8][0]
            arrX1[i]=arrsadcount[i-8][1]
            arrX2[i]=arrsadcount[i-8][2]
            arrX3[i]=arrsadcount[i-8][3]
            arrX4[i]=arrsadcount[i-8][4]
            arrX5[i]=arrsadcount[i-8][5]
            arrX6[i]=arrsadcount[i-8][6]
            arrX7[i]=arrsadcount[i-8][7]
            arrX8[i]=arrsadcount[i-8][8]
    print("arrX0=",arrX0)
    print("arrX1=",arrX1)
    print("arrX2=",arrX2)
    print("arrX3=",arrX3)
    print("arrX4=",arrX4)
    print("arrX5=",arrX5)
    print("arrX6=",arrX6)
    print("arrX7=",arrX7)
    print("arrX8=",arrX8)
    y=[1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0]

    #計算準確率
    arra = [0, 0, 0, 0, 0, 0, 0, 0, 0]
    number = '{:09b}'.format(n, "b")
    for p in range(0,9):
        arra[p]=int(number[p])
        print(arra[p],end='')
    test = []
    if arra[0] == 1:
        if len(test):
            test = test
        else:
            test = np.c_[arrX0]
    if arra[1] == 1:
        if len(test):
            test = np.c_[test, arrX1]
        else:
            test = np.c_[arrX1]
    if arra[2] == 1:
        if len(test):
            test = np.c_[test, arrX2]
        else:
            test = np.c_[arrX2]
    if arra[3] == 1:
        if len(test):
            test = np.c_[test, arrX3]
        else:
            test = np.c_[arrX3]
    if arra[4] == 1:
        if len(test):
            test = np.c_[test, arrX4]
        else:
            test = np.c_[arrX4]
    if arra[5] == 1:
        if len(test):
            test = np.c_[test, arrX5]
        else:
            test = np.c_[arrX5]
    if arra[6] == 1:
        if len(test):
            test = np.c_[test,arrX6]
        else:
            test = np.c_[arrX6]
    if arra[7] == 1:
        if len(test):
            test = np.c_[test, arrX7]
        else:
            test = np.c_[arrX7]
    if arra[8] == 1:
        if len(test):
            test = np.c_[test, arrX8]
        else:
            test = np.c_[arrX8]

    arra[8] += 1
    X_train, X_test, y_train, y_test = train_test_split(test, y,test_size=0.3,random_state=1)
    maxScore = 0
    kn = 0
    for o in range(1, 8):
        clf = KNeighborsClassifier(n_neighbors = o,p = 2,weights = 'distance',algorithm = 'auto')
        clf.fit(X_train,y_train)
        score = clf.score(X_test,y_test)
        if score > maxScore:
            maxScore = score
            kn = o
    print()
    print(maxScore,kn)
    return jsonify(maxScore)
if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Oct/2021 17:21:03] "GET / HTTP/1.1" 404 -


[1000, 86, 85, 78, 77, 75, 74, 73, 72, 74]
[84, 88, 89, 91, 86, 86, 83, 83, 84, 61]
[84, 76, 77, 80, 83, 86, 84, 86, 85, 70]
[[1000, 86, 85], [86, 85, 78], [85, 78, 77], [78, 77, 75], [77, 75, 74], [75, 74, 73], [74, 73, 72], [73, 72, 74]]
[[84, 88, 89], [88, 89, 91], [89, 91, 86], [91, 86, 86], [86, 86, 83], [86, 83, 83], [83, 83, 84], [83, 84, 61]]
[[84, 76, 77], [76, 77, 80], [77, 80, 83], [80, 83, 86], [83, 86, 84], [86, 84, 86], [84, 86, 85], [86, 85, 70]]
0
happy心率特徵 :
平均:  87.0
一階差:  2.5
二階差:  5.0
最大心率:  89
最小心率:  84
心率變化範圍:  5
資訊熵:  1.584962500721156
均方根:  5.666666666666667
標準化均值:  -303.3333333333333
[(87.0, 2.5, 5.0, 89, 84, 5, 1.584962500721156, 5.666666666666667, -303.3333333333333), 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
subject心率特徵: 
平均:  79.0
一階差:  4.5
二階差:  7.0
最大心率:  84
最小心率:  76
心率變化範圍:  8
資訊熵:  1.584962500721156
均方根:  21.666666666666668
標準化均值:  -311.3333333333333
1
happy心率特徵 :
平均:  89.33333333333333
一階差:  1.5
二階差:  3.0
最大心率:  91
最小心率:  88
心率變化範圍:  3
資訊熵:  1.58496250072115

127.0.0.1 - - [14/Oct/2021 17:21:15] "GET /yu/設計群測量/336 HTTP/1.1" 200 -


0.6 1


127.0.0.1 - - [14/Oct/2021 17:21:20] "GET /yu/設計群測量/1 HTTP/1.1" 200 -


[1000, 86, 85, 78, 77, 75, 74, 73, 72, 74]
[84, 88, 89, 91, 86, 86, 83, 83, 84, 61]
[84, 76, 77, 80, 83, 86, 84, 86, 85, 70]
[[1000, 86, 85], [86, 85, 78], [85, 78, 77], [78, 77, 75], [77, 75, 74], [75, 74, 73], [74, 73, 72], [73, 72, 74]]
[[84, 88, 89], [88, 89, 91], [89, 91, 86], [91, 86, 86], [86, 86, 83], [86, 83, 83], [83, 83, 84], [83, 84, 61]]
[[84, 76, 77], [76, 77, 80], [77, 80, 83], [80, 83, 86], [83, 86, 84], [86, 84, 86], [84, 86, 85], [86, 85, 70]]
0
happy心率特徵 :
平均:  87.0
一階差:  2.5
二階差:  5.0
最大心率:  89
最小心率:  84
心率變化範圍:  5
資訊熵:  1.584962500721156
均方根:  5.666666666666667
標準化均值:  -303.3333333333333
[(87.0, 2.5, 5.0, 89, 84, 5, 1.584962500721156, 5.666666666666667, -303.3333333333333), 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
subject心率特徵: 
平均:  79.0
一階差:  4.5
二階差:  7.0
最大心率:  84
最小心率:  76
心率變化範圍:  8
資訊熵:  1.584962500721156
均方根:  21.666666666666668
標準化均值:  -311.3333333333333
1
happy心率特徵 :
平均:  89.33333333333333
一階差:  1.5
二階差:  3.0
最大心率:  91
最小心率:  88
心率變化範圍:  3
資訊熵:  1.58496250072115

127.0.0.1 - - [14/Oct/2021 17:21:26] "GET /yu/設計群測量/511 HTTP/1.1" 200 -


[1000, 86, 85, 78, 77, 75, 74, 73, 72, 74]
[84, 88, 89, 91, 86, 86, 83, 83, 84, 61]
[84, 76, 77, 80, 83, 86, 84, 86, 85, 70]
[[1000, 86, 85], [86, 85, 78], [85, 78, 77], [78, 77, 75], [77, 75, 74], [75, 74, 73], [74, 73, 72], [73, 72, 74]]
[[84, 88, 89], [88, 89, 91], [89, 91, 86], [91, 86, 86], [86, 86, 83], [86, 83, 83], [83, 83, 84], [83, 84, 61]]
[[84, 76, 77], [76, 77, 80], [77, 80, 83], [80, 83, 86], [83, 86, 84], [86, 84, 86], [84, 86, 85], [86, 85, 70]]
0
happy心率特徵 :
平均:  87.0
一階差:  2.5
二階差:  5.0
最大心率:  89
最小心率:  84
心率變化範圍:  5
資訊熵:  1.584962500721156
均方根:  5.666666666666667
標準化均值:  -303.3333333333333
[(87.0, 2.5, 5.0, 89, 84, 5, 1.584962500721156, 5.666666666666667, -303.3333333333333), 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
subject心率特徵: 
平均:  79.0
一階差:  4.5
二階差:  7.0
最大心率:  84
最小心率:  76
心率變化範圍:  8
資訊熵:  1.584962500721156
均方根:  21.666666666666668
標準化均值:  -311.3333333333333
1
happy心率特徵 :
平均:  89.33333333333333
一階差:  1.5
二階差:  3.0
最大心率:  91
最小心率:  88
心率變化範圍:  3
資訊熵:  1.58496250072115